In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import time

In [2]:
executable_path = {'executable_path': 'chromedriver'}
browser = Browser('chrome', **executable_path, headless=True)

In [3]:
url_1 = 'https://mars.nasa.gov/news/'
browser.visit(url_1)
soup_1 = BeautifulSoup(browser.html, "html.parser")
news_title = soup_1.find('div', class_='content_title').find('a').text
news_p = soup_1.find('div', class_='article_teaser_body').text

In [4]:
url_2 = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_2)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(5)
browser.click_link_by_partial_text('more info')
soup_2 = BeautifulSoup(browser.html, "html.parser")
featured_image_url = "https://www.jpl.nasa.gov{}".format(soup_2.find('figure', class_='lede').a['href'])

In [5]:
url_3 = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_3)
soup_3 = BeautifulSoup(browser.html, 'html.parser')
mars_tweets = soup_3.find_all('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")
for mars_tweet in mars_tweets:
    first_word = mars_tweet.text.split(" ")[0]
    if first_word == "Sol":
        mars_weather = mars_tweet.text
        break

In [6]:
url_4 = "https://space-facts.com/mars/"
browser.visit(url_4)
soup_4 = BeautifulSoup(browser.html, 'html.parser')
facts_data = soup_4.find('table', class_="tablepress tablepress-id-mars")
df_facts = pd.read_html(str(facts_data))
facts_table = df_facts[0].to_html(index=False, header=None, escape=True).replace("\n", "")

In [7]:
url_5 = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_5)
soup_5 = BeautifulSoup(browser.html, "html.parser")
facts_links = soup_5.find('div', class_="collapsible results").find_all('a', href=True)
hemisphere_image_urls = []
for facts_link in facts_links:
    r = requests.get('https://astrogeology.usgs.gov'+facts_link['href'])
    soup_5_temp = BeautifulSoup(r.content)
    facts_title = soup_5_temp.find('title').text.split(' Enhanced')[0]
    facts_img = 'https://astrogeology.usgs.gov' + soup_5_temp.find('img',class_='wide-image').get('src')
    hemisphere_image_urls.append({'title': facts_title, 'img_url':facts_img})

C:\Users\jamie\Anaconda3\envs\PythonData\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\jamie\Anaconda3\envs\PythonData\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [8]:
final = {
        "id": 0,
        "news_title": news_title,
        "news_p": news_p,
        "featured_image": featured_image_url,
        "mars_weather": mars_weather,
        "facts_table": facts_table,
        "hemisphere_images": hemisphere_image_urls
    }

In [16]:
final['hemisphere_images']

[{'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere'}]